# FinRisk: Credit Risk & Fraud Detection - Notebook 2
## Phase 1: Data Preprocessing & Feature Engineering

**Objective:** To clean the raw datasets, handle missing values, correct data types, and engineer new features to enhance model performance for both credit risk and fraud detection.

In [15]:
# ==============================================================================
# 1. Import Libraries
# ==============================================================================
import pandas as pd
import numpy as np
import os

# Configure settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


## 2. Load Datasets

Loading the raw data from the `../data/raw/` directory.

In [16]:
# ==============================================================================
# 2. Load Datasets
# ==============================================================================
# Define file paths based on the project structure
RAW_DATA_PATH = '../data/raw/'
PROCESSED_DATA_PATH = '../data/processed/'

# Create processed data directory if it doesn't exist
if not os.path.exists(PROCESSED_DATA_PATH):
    os.makedirs(PROCESSED_DATA_PATH)

print("--- Loading Datasets ---")

try:
    credit_applications = pd.read_csv(os.path.join(RAW_DATA_PATH, 'credit_applications.csv'))
    transaction_data = pd.read_csv(os.path.join(RAW_DATA_PATH, 'transaction_data.csv'))
    customer_profiles = pd.read_csv(os.path.join(RAW_DATA_PATH, 'customer_profiles.csv'))
    credit_bureau_data = pd.read_csv(os.path.join(RAW_DATA_PATH, 'credit_bureau_data.csv'))
    
    print("All datasets loaded successfully.")
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}. Please ensure the CSV files are in the '../data/raw/' directory.")
    assert False, "Dataset files not found."

--- Loading Datasets ---
All datasets loaded successfully.


## 3. Data Cleaning & Preprocessing

In this step, we will:
1.  Convert date columns to the correct `datetime` format.
2.  Handle missing values using appropriate strategies (e.g., imputation).
3.  Standardize formats where necessary.

In [17]:
print("--- Cleaning: Credit Applications ---")
# Convert date columns
credit_applications['application_date'] = pd.to_datetime(credit_applications['application_date'], format='mixed', dayfirst=True)

# Check for missing values - for this project,never assume the data is clean.
# In a some scenario, we'd implement imputation strategies here.
# For example: credit_applications['debt_to_income_ratio'].fillna(credit_applications['debt_to_income_ratio'].median(), inplace=True)
print(credit_applications.isnull().sum())
print("\n'application_date' converted to datetime.")


print("\n--- Cleaning: Transaction Data ---")
# Convert date columns
transaction_data['transaction_date'] = pd.to_datetime(transaction_data['transaction_date'], format='mixed', dayfirst=True)
print(transaction_data.isnull().sum())
print("\n'transaction_date' converted to datetime.")


print("\n--- Cleaning: Customer Profiles ---")
# Convert date columns
customer_profiles['last_activity_date'] = pd.to_datetime(customer_profiles['last_activity_date'], format='mixed', dayfirst=True)
print(customer_profiles.isnull().sum())
print("\n'last_activity_date' converted to datetime.")

--- Cleaning: Credit Applications ---
application_id          0
customer_id             0
application_date        0
loan_amount             0
loan_purpose            0
employment_status       0
annual_income           0
debt_to_income_ratio    0
credit_score            0
application_status      0
default_flag            0
dtype: int64

'application_date' converted to datetime.

--- Cleaning: Transaction Data ---
transaction_id          0
customer_id             0
transaction_date        0
amount                  0
merchant_category       0
transaction_type        0
location                0
device_info             0
fraud_flag              0
investigation_status    0
dtype: int64

'transaction_date' converted to datetime.

--- Cleaning: Customer Profiles ---
customer_id           0
customer_age          0
annual_income         0
employment_status     0
account_tenure        0
product_holdings      0
relationship_value    0
risk_segment          0
behavioral_score      0
credit_score   

## 4. Feature Engineering for Credit Risk Modeling

We will create a master dataset for credit risk by merging the relevant tables and engineering new features.

In [18]:
# ==============================================================================
# 4. Feature Engineering for Credit Risk Modeling
# ==============================================================================
# --- Merge Datasets for Credit Risk ---
print("\n--- Merging datasets for credit risk model ---")

# Merge applications with customer profiles, adding suffixes
credit_df = pd.merge(
    credit_applications, 
    customer_profiles, 
    on='customer_id', 
    how='left', 
    suffixes=('_app', '_profile')
)

# Merge the result with credit bureau data
# The suffix for the new conflicting column 'credit_score' will be '_bureau'
credit_df = pd.merge(
    credit_df, 
    credit_bureau_data, 
    on='customer_id', 
    how='left', 
    suffixes=('', '_bureau') # No suffix for left df, _bureau for right
)


print(f"Merged credit risk dataset shape: {credit_df.shape}")
print("Columns after merge:", credit_df.columns.tolist())




--- Merging datasets for credit risk model ---
Merged credit risk dataset shape: (100000, 29)
Columns after merge: ['application_id', 'customer_id', 'application_date', 'loan_amount', 'loan_purpose', 'employment_status_app', 'annual_income_app', 'debt_to_income_ratio', 'credit_score_app', 'application_status', 'default_flag', 'customer_age', 'annual_income_profile', 'employment_status_profile', 'account_tenure', 'product_holdings', 'relationship_value', 'risk_segment', 'behavioral_score', 'credit_score_profile', 'city', 'last_activity_date', 'credit_score', 'credit_history_length', 'number_of_accounts', 'total_credit_limit', 'credit_utilization', 'payment_history', 'public_records']


## Feature Engineering:

1. loan_to_income_ratio (loan_amount / annual_income) : This feature captures that relative risk.
2. days_since_last_activity (analysis_date - last_activity_dat) :  A recent last activity date suggests an active and engaged customer. A long period of inactivity could be a subtle risk factor, indicating the customer may be moving their primary banking elsewhere.
3. credit_history_years	(credit_history_length (months) / 12) : Converting months to years makes the feature more intuitive for analysis and for explaining model decisions to stakeholders.
4. credit_score_x_dti(credit_score * debt_to_income_ratio) :  This feature captures the combined effect of creditworthiness and existing debt. A person with a decent credit score might still be high-risk if their DTI is also very high. This interaction feature helps the model explicitly learn this complex relationship.

In [19]:
# --- Create New Features ---
print("\n--- Engineering new features for credit risk ---")

# Financial Ratios (using profile data as the source of truth)
credit_df['loan_to_income_ratio'] = credit_df['loan_amount'] / credit_df['annual_income_profile']

# Time-based Features
analysis_date = credit_df['application_date'].max()
credit_df['days_since_last_activity'] = (analysis_date - credit_df['last_activity_date']).dt.days
credit_df['credit_history_years'] = credit_df['credit_history_length'] / 12

# Interaction Terms
credit_df['credit_score_x_dti'] = credit_df['credit_score_profile'] * credit_df['debt_to_income_ratio']

# Handle potential infinite values from division by zero
credit_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# We can now decide on a strategy for potentially missing data. For now, fill with 0.
credit_df.fillna(0, inplace=True)

print("\nNew features created: 'loan_to_income_ratio', 'days_since_last_activity', 'credit_history_years', 'credit_score_x_dti'")
print("\nSample of the enhanced credit risk data:")
print(credit_df.head())
print("Columns after merge:", credit_df.columns.tolist())


--- Engineering new features for credit risk ---

New features created: 'loan_to_income_ratio', 'days_since_last_activity', 'credit_history_years', 'credit_score_x_dti'

Sample of the enhanced credit risk data:
  application_id  customer_id application_date  loan_amount   loan_purpose  \
0    APP_0000001  CUST_008172       2023-09-03       164883  Home Purchase   
1    APP_0000002  CUST_016851       2023-05-02        59782  Home Purchase   
2    APP_0000003  CUST_016766       2024-08-08        23716   Car Purchase   
3    APP_0000004  CUST_018679       2024-07-22        13514   Car Purchase   
4    APP_0000005  CUST_019411       2024-08-13        13992   Car Purchase   

  employment_status_app  annual_income_app  debt_to_income_ratio  \
0               Retired           85399.18                 0.828   
1             Full-time           20582.50                 1.005   
2             Part-time           11860.21                 0.694   
3             Part-time           16490.54     

 # Processed Data Cleaning

In [20]:
# Processed data 'credit_df' cleaning
# drop duplicates 'drop -annual_income_app', 'credit_score_app'
credit_df = credit_df.drop(columns=['annual_income_app', 'credit_score_app', 'employment_status_app','credit_score_profile'])

# remove outliers from 'customer_age'
Q1 = credit_df['customer_age'].quantile(0.25)
Q3 = credit_df['customer_age'].quantile(0.75)
IQR = Q3 - Q1
credit_df = credit_df[(credit_df['customer_age'] >= Q1 - 1.5 * IQR) & (credit_df['customer_age'] <= Q3 + 1.5 * IQR)]


# reset index
credit_df = credit_df.reset_index(drop=True)

# print cleaned DataFrame
print(credit_df.head())

print(credit_df.describe())

  application_id  customer_id application_date  loan_amount   loan_purpose  \
0    APP_0000001  CUST_008172       2023-09-03       164883  Home Purchase   
1    APP_0000002  CUST_016851       2023-05-02        59782  Home Purchase   
2    APP_0000003  CUST_016766       2024-08-08        23716   Car Purchase   
3    APP_0000004  CUST_018679       2024-07-22        13514   Car Purchase   
4    APP_0000005  CUST_019411       2024-08-13        13992   Car Purchase   

   debt_to_income_ratio application_status  default_flag  customer_age  \
0                 0.828           Declined             0            81   
1                 1.005           Declined             0            34   
2                 0.694           Declined             0            18   
3                 0.710           Declined             0            18   
4                 0.530           Declined             0            18   

   annual_income_profile employment_status_profile  account_tenure  \
0               

## 5. Feature Engineering for Fraud Detection

For fraud detection, we'll aggregate transaction data to create behavioral features for each customer.

In [21]:
# ==============================================================================
# 5. Feature Engineering for Fraud Detection 
# ==============================================================================
print("\n--- Engineering features for fraud detection ---")

transaction_data['transaction_hour'] = transaction_data['transaction_date'].dt.hour
transaction_data['transaction_day_of_week'] = transaction_data['transaction_date'].dt.dayofweek

customer_fraud_features = transaction_data.groupby('customer_id').agg(
    avg_transaction_amount=('amount', 'mean'),
    std_transaction_amount=('amount', 'std'),
    total_transactions=('transaction_id', 'count'),
    total_transaction_value=('amount', 'sum'),
    num_fraud_flags=('fraud_flag', 'sum')
).reset_index()

customer_fraud_features.fillna(0, inplace=True)
print("Customer-level fraud features created.")
print("\nSample of the fraud features data:")
print(customer_fraud_features.head())



--- Engineering features for fraud detection ---
Customer-level fraud features created.

Sample of the fraud features data:
   customer_id  avg_transaction_amount  std_transaction_amount  \
0  CUST_000001               50.684000               34.139959   
1  CUST_000002               54.605000               31.304545   
2  CUST_000003               77.072000               98.179513   
3  CUST_000004              995.358571             2266.905929   
4  CUST_000005              229.306667              292.077668   

   total_transactions  total_transaction_value  num_fraud_flags  
0                   5                   253.42                0  
1                   4                   218.42                0  
2                   5                   385.36                0  
3                   7                  6967.51                1  
4                   6                  1375.84                0  


## 6. Save Processed Data

Saving the newly created dataframes to the `processed` directory for use in the modeling phases.

In [22]:
# ==============================================================================
# 6. Save Processed Data
# ==============================================================================
# Define output file paths
credit_risk_output_path = os.path.join(PROCESSED_DATA_PATH, 'credit_risk_features.csv')
fraud_detection_output_path = os.path.join(PROCESSED_DATA_PATH, 'fraud_detection_features.csv')
transaction_data_output_path = os.path.join(PROCESSED_DATA_PATH, 'transaction_data_enhanced.csv')

# Save the dataframes
credit_df.to_csv(credit_risk_output_path, index=False)
print(f"\nProcessed credit risk data saved to: {credit_risk_output_path}")

customer_fraud_features.to_csv(fraud_detection_output_path, index=False)
print(f"Processed fraud detection features saved to: {fraud_detection_output_path}")

transaction_data.to_csv(transaction_data_output_path, index=False)
print(f"Enhanced transaction data saved to: {transaction_data_output_path}")

print("\n" + "="*80)
print("End of Data Preprocessing and Feature Engineering Script")
print("="*80)


Processed credit risk data saved to: ../data/processed/credit_risk_features.csv
Processed fraud detection features saved to: ../data/processed/fraud_detection_features.csv
Enhanced transaction data saved to: ../data/processed/transaction_data_enhanced.csv

End of Data Preprocessing and Feature Engineering Script


# Correlation Analysis/Feature Selections:

In [23]:
df = pd.read_csv('../data/processed/credit_risk_features.csv')
df.head()

,application_id,customer_id,application_date,loan_amount,loan_purpose,debt_to_income_ratio,application_status,default_flag,customer_age,annual_income_profile,employment_status_profile,account_tenure,product_holdings,relationship_value,risk_segment,behavioral_score,city,last_activity_date,credit_score,credit_history_length,number_of_accounts,total_credit_limit,credit_utilization,payment_history,public_records,loan_to_income_ratio,days_since_last_activity,credit_history_years,credit_score_x_dti
0,APP_0000001,CUST_008172,2023-09-03,164883,Home Purchase,0.828,Declined,0,81,85399.18,Retired,1,3,28181.73,Near-Prime,150.71,London,2025-07-13,675,14,5,22286,0.633,0.865,0,1.930733,-195,1.166667,558.900
1,APP_0000002,CUST_016851,2023-05-02,59782,Home Purchase,1.005,Declined,0,34,20582.50,Full-time,2,3,7409.70,Near-Prime,440.25,Glasgow,2025-08-03,677,3,2,13976,0.611,0.884,0,2.904506,-216,0.250000,680.385
2,APP_0000003,CUST_016766,2024-08-08,23716,Car Purchase,0.694,Declined,0,18,11860.21,Part-time,1,2,2609.25,Near-Prime,328.65,London,2025-07-27,732,0,2,1695,0.480,0.888,0,1.999627,-209,0.000000,508.008
3,APP_0000004,CUST_018679,2024-07-22,13514,Car Purchase,0.710,Declined,0,18,16490.54,Part-time,7,3,8410.18,Prime,207.90,London,2025-08-05,850,0,6,1530,0.021,0.995,0,0.819500,-218,0.000000,603.500
4,APP_0000005,CUST_019411,2024-08-13,13992,Car Purchase,0.530,Declined,0,18,9600.00,Part-time,4,1,1344.00,Subprime,478.07,London,2025-07-27,630,0,6,7076,0.643,0.809,0,1.457500,-209,0.000000,333.900


In [24]:
print(df.shape)
df.describe()

(96766, 29)


,loan_amount,debt_to_income_ratio,default_flag,customer_age,annual_income_profile,account_tenure,product_holdings,relationship_value,behavioral_score,credit_score,credit_history_length,number_of_accounts,total_credit_limit,credit_utilization,payment_history,public_records,loan_to_income_ratio,days_since_last_activity,credit_history_years,credit_score_x_dti
count,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000,96766.000000
mean,86080.515119,0.737155,0.019511,38.039539,43098.433716,3.506573,2.587779,16765.101940,284.502205,732.614865,6.875152,3.856427,15669.440764,0.345812,0.902718,0.021536,1.936060,-205.491009,0.572929,545.543065
std,85273.828217,0.615897,0.138313,21.161428,31224.119001,2.831815,1.253717,17513.145859,157.640268,86.330034,9.632454,2.183842,18468.727869,0.237503,0.096378,0.165147,2.994533,8.677425,0.802705,463.511613
min,1000.000000,0.203000,0.000000,18.000000,2.590000,0.000000,1.000000,0.540000,2.190000,379.000000,0.000000,0.000000,500.000000,0.000000,0.600000,0.000000,0.500020,-220.000000,0.000000,103.936000
25%,24156.000000,0.557000,0.000000,18.000000,16000.000000,1.000000,2.000000,4058.007500,162.290000,673.000000,0.000000,2.000000,3602.000000,0.145000,0.869000,0.000000,1.142140,-213.000000,0.000000,389.180000
50%,56837.500000,0.718000,0.000000,32.000000,37330.900000,3.000000,2.000000,10500.860000,263.470000,738.000000,3.000000,4.000000,9247.000000,0.307000,0.937000,0.000000,1.769894,-205.000000,0.250000,516.614000
75%,120991.750000,0.891000,0.000000,50.000000,62371.790000,5.000000,3.000000,23830.407500,385.960000,804.000000,10.000000,5.000000,20702.000000,0.517000,0.972000,0.000000,2.575792,-198.000000,0.833333,675.324000
max,670207.000000,77.791000,1.000000,104.000000,210598.470000,24.000000,5.000000,170424.580000,927.240000,850.000000,100.000000,15.000000,193487.000000,0.992000,1.000000,3.000000,386.100386,-191.000000,8.333333,57409.758000


In [25]:
df1 = pd.read_csv('../data/processed/fraud_detection_features.csv')
df1.head()


,customer_id,avg_transaction_amount,std_transaction_amount,total_transactions,total_transaction_value,num_fraud_flags
0,CUST_000001,50.684000,34.139959,5,253.42,0
1,CUST_000002,54.605000,31.304545,4,218.42,0
2,CUST_000003,77.072000,98.179513,5,385.36,0
3,CUST_000004,995.358571,2266.905929,7,6967.51,1
4,CUST_000005,229.306667,292.077668,6,1375.84,0


In [26]:
print(df1.shape)
df1.describe(include="all")

(24541, 6)


,customer_id,avg_transaction_amount,std_transaction_amount,total_transactions,total_transaction_value,num_fraud_flags
count,24541,24541.000000,24541.000000,24541.000000,24541.000000,24541.000000
unique,24541,NaN,NaN,NaN,NaN,NaN
top,CUST_025000,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN
mean,NaN,114.592376,94.920519,4.074814,469.664469,0.005664
std,NaN,166.773603,186.878634,1.930882,732.781503,0.075589
min,NaN,1.000000,0.000000,1.000000,1.000000,0.000000
25%,NaN,26.440000,12.320491,3.000000,87.390000,0.000000
50%,NaN,63.703333,40.128310,4.000000,235.300000,0.000000
75%,NaN,139.160000,104.966227,5.000000,553.290000,0.000000


In [27]:
df2 = pd.read_csv('../data/processed/transaction_data_enhanced.csv')
df2.head()

,transaction_id,customer_id,transaction_date,amount,merchant_category,transaction_type,location,device_info,fraud_flag,investigation_status,transaction_hour,transaction_day_of_week
0,TXN_00000001,CUST_004019,2024-03-25 11:24:31,34.85,Groceries,Purchase,Manchester,Mobile,0,Not Investigated,11,0
1,TXN_00000002,CUST_000222,2022-05-24 14:21:01,84.32,Fuel,Purchase,Bristol,Mobile,0,Not Investigated,14,1
2,TXN_00000003,CUST_007895,2023-01-15 10:15:41,6.82,Fuel,Purchase,London,Chip,0,Not Investigated,10,6
3,TXN_00000004,CUST_021148,2023-10-03 06:15:45,59.90,Groceries,Purchase,Birmingham,Contactless,0,Not Investigated,6,1
4,TXN_00000005,CUST_016591,2024-11-24 09:09:02,71.01,Transfer,Transfer,Leeds,Desktop,0,Not Investigated,9,6


In [28]:
print(df2.shape)
df2.describe(include="all")

(100000, 12)


,transaction_id,customer_id,transaction_date,amount,merchant_category,transaction_type,location,device_info,fraud_flag,investigation_status,transaction_hour,transaction_day_of_week
count,100000,100000,100000,100000.000000,100000,100000,100000,100000,100000.000000,100000,100000.00000,100000.000000
unique,100000,24541,99940,NaN,10,3,10,5,NaN,4,NaN,NaN
top,TXN_00099984,CUST_018784,2024-08-26 00:36:51,NaN,Groceries,Purchase,Glasgow,Chip,NaN,Not Investigated,NaN,NaN
freq,1,14,2,NaN,20650,94025,10366,20268,NaN,99861,NaN,NaN
mean,NaN,NaN,NaN,115.260357,NaN,NaN,NaN,NaN,0.001390,NaN,11.44186,3.009540
std,NaN,NaN,NaN,250.922703,NaN,NaN,NaN,NaN,0.037257,NaN,6.92140,2.001197
min,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.000000,NaN,0.00000,0.000000
25%,NaN,NaN,NaN,16.460000,NaN,NaN,NaN,NaN,0.000000,NaN,5.00000,1.000000
50%,NaN,NaN,NaN,45.370000,NaN,NaN,NaN,NaN,0.000000,NaN,11.00000,3.000000
75%,NaN,NaN,NaN,118.050000,NaN,NaN,NaN,NaN,0.000000,NaN,17.00000,5.000000
